## Scrape Worldometer with Bs4

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, date


In [25]:
def worldometer_scrape(days_from_today):
    """Scrape covid statistics from worldometer.

    Input por days_from_today are =0 for today, 1 for yesterday.

    """
    url = 'https://www.worldometers.info/coronavirus/'

    page = requests.get(url)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    soup_table = soup.findAll('table')[days_from_today]

    header = soup_table.findAll('thead')[0]

    header_list = header.findAll('th')
    colnames = []
    for i in header_list:
        colnames.append(i.getText())

    rows = []
    # tbody is where the table is, tr its the row, td its the  cell inside the row
    body = soup_table.findAll('tbody')[0]
    body_list = body.findAll('tr')
    for i in body_list:
        row = []
        body_row = i.findAll('td')
        for j in body_row:
            row.append(j.getText().strip())
        rows.append(row)

    df = pd.DataFrame(rows, columns=colnames)
    df = df.rename(columns={"Country,Other": "country",
                            "Tests/\n1M pop\n": "Test/1M",
                            "Tot\xa0Cases/1M pop": "Tot Cases/1M",
                            "Deaths/1M pop": "Deaths/1M"}, errors="raise")
    df['TotalDeaths'] = df['TotalDeaths'].str.replace(",", "")
    df['TotalDeaths'] = pd.to_numeric(df['TotalDeaths'])
    df['Deaths/1M'] = df['Deaths/1M'].str.replace(",", "")
    df['Deaths/1M'] = pd.to_numeric(df['Deaths/1M'])
    df['Test/1M'] = df['Test/1M'].str.replace(",", "")
    df['Test/1M'] = pd.to_numeric(df['Test/1M'])
    
    df_final = df[['country', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
      'TotalTests', 'Population']]
    
  
    return df_final

In [27]:
df_today = worldometer_scrape(0)

#
Country,Other
TotalCases
NewCases
TotalDeaths
NewDeaths
TotalRecovered
NewRecovered
ActiveCases
Serious,Critical
Tot Cases/1M pop
Deaths/1M pop
TotalTests
Tests/
1M pop

Population
Continent
1 Caseevery X ppl
1 Deathevery X ppl
1 Testevery X ppl
New Cases/1M pop
New Deaths/1M pop
Active Cases/1M pop


In [28]:
df_today.head(15)

,country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,Population
0,North America,"55,053,428","+2,005",1119945.0,+60,"43,344,194","+3,931","10,589,289","22,307",,
1,Asia,"77,959,744","+26,734",1149476.0,+304,"74,907,639","+47,106","1,902,629","28,863",,
2,South America,"38,147,667",+177,1164243.0,+2,"36,282,162",+511,"701,262","12,569",,
3,Europe,"61,599,969","+55,809",1262492.0,"+1,278","56,192,814","+36,603","4,144,663","12,003",,
4,Africa,"8,505,243",,216054.0,,"7,826,953",,"462,236","2,536",,
5,Oceania,"276,844","+2,336",3383.0,+12,"216,557",+54,"56,904",373,,
6,,721,,15.0,,706,,0,0,,
7,World,"241,543,616","+87,061",4915608.0,"+1,656","218,771,025","+88,205","17,856,983","78,651",,
8,USA,"45,792,532",,744546.0,,"35,374,595",,"9,673,391","15,133","668,769,691","333,511,317"
9,India,"34,081,315",+266,452321.0,,"33,439,331","+7,615","189,663","8,944","591,924,874","1,397,531,610"
